# Using TensorFlow Scripts in SageMaker - Quickstart

Starting with TensorFlow version 1.11, you can use SageMaker's TensorFlow containers to train TensorFlow scripts the same way you would train outside SageMaker. This feature is named **Script Mode**. 

This example uses 
[Multi-layer Recurrent Neural Networks (LSTM, RNN) for character-level language models in Python using Tensorflow](https://github.com/sherjilozair/char-rnn-tensorflow). 
You can use the same technique for other scripts or repositories, including 
[TensorFlow Model Zoo](https://github.com/tensorflow/models) and 
[TensorFlow benchmark scripts](https://github.com/tensorflow/benchmarks/tree/master/scripts/tf_cnn_benchmarks).

## Test locally using SageMaker Python SDK TensorFlow Estimator

You can use the SageMaker Python SDK [`TensorFlow`](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/README.rst#training-with-tensorflow) estimator to easily train locally and in SageMaker. 

Let's start by setting the training script arguments `--num_epochs` and `--data_dir` as hyperparameters. Remember that we don't need to provide `--model_dir`:

In [1]:
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
hyperparameters = {'task_name': 'ner', 'do_train': 'true', 'do_eval': 'true', 
                   'data_dir': '/opt/ml/input/data/training', 'output_dir': '/opt/ml/model', 
                   'vocab_file': './albert_config/vocab.txt', 'bert_config_file': './albert_base_zh/albert_config_base.json', 
                   'max_seq_length': 128, 'train_batch_size': 64, 'learning_rate': 2e-5, 'num_train_epochs': 1}

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments. Just change your estimator's train_instance_type to local or local_gpu. For more information, see: https://github.com/aws/sagemaker-python-sdk#local-mode.

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU). Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

Note, you can only run a single local notebook at a time.

In [3]:
!/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


To train locally, you set `train_instance_type` to [local](https://github.com/aws/sagemaker-python-sdk#local-mode):

In [4]:
import subprocess

train_instance_type='local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    train_instance_type = 'local_gpu'
    
print("Train instance type = " + train_instance_type)

Train instance type = local_gpu


We create the `TensorFlow` Estimator, passing the `git_config` argument and the flag `script_mode=True`. Note that we are using Git integration here, so `source_dir` should be a relative path inside the Git repo; otherwise it should be a relative or absolute local path. the `Tensorflow` Estimator is created as following: 


In [5]:
import os

import sagemaker
from sagemaker.tensorflow import TensorFlow


estimator = TensorFlow(entry_point='albert_ner.py',
                       source_dir='.',
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.15.2',
                       py_version='py3',
                       script_mode=True)

To start a training job, we call `estimator.fit(inputs)`, where inputs is a dictionary where the keys, named **channels**, 
have values pointing to the data location. `estimator.fit(inputs)` downloads the TensorFlow container with TensorFlow Python 3, CPU version, locally and simulates a SageMaker training job. 
When training starts, the TensorFlow container executes **train.py**, passing `hyperparameters` and `model_dir` as script arguments, executing the example as follows:
```bash
python -m train --num-epochs 1 --data_dir /opt/ml/input/data/training --model_dir /opt/ml/model
```


In [6]:
inputs = {'training': f'file:///home/ec2-user/SageMaker/albert-chinese-ner/data/'}

estimator.fit(inputs)

Creating tmp_z1qkpgr_algo-1-5us5l_1 ... 
Attaching to tmp_z1qkpgr_algo-1-5us5l_12mdone
algo-1-5us5l_1  | WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
algo-1-5us5l_1  | 
algo-1-5us5l_1  | 2020-05-28 10:28:20,359 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-5us5l_1  | 2020-05-28 10:28:24,179 sagemaker-containers INFO     Invoking user script
algo-1-5us5l_1  | 
algo-1-5us5l_1  | Training Env:
algo-1-5us5l_1  | 
algo-1-5us5l_1  | {
algo-1-5us5l_1  |     "additional_framework_parameters": {},
algo-1-5us5l_1  |     "channel_input_dirs": {
algo-1-5us5l_1  |         "training": "/opt/ml/input/data/training"
algo-1-5us5l_1  |     },
algo-1-5us5l_1  |     "current_host": "algo-1-5us5l",
algo-1-5us5l_1  |     "framework_module": "sagemaker_tensorflow_container.training:main",
algo-1-5us5l_1  |     "host

Let's explain the values of `--data_dir` and `--model_dir` with more details:

- **/opt/ml/input/data/training** is the directory inside the container where the training data is downloaded. The data is downloaded to this folder because `training` is the channel name defined in ```estimator.fit({'training': inputs})```. See [training data](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html#your-algorithms-training-algo-running-container-trainingdata) for more information. 

- **/opt/ml/model** use this directory to save models, checkpoints, or any other data. Any data saved in this folder is saved in the S3 bucket defined for training. See [model data](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html#your-algorithms-training-algo-envvariables) for more information.

### Reading additional information from the container

Often, a user script needs additional information from the container that is not available in ```hyperparameters```.
SageMaker containers write this information as **environment variables** that are available inside the script.

For example, the example above can read information about the `training` channel provided in the training job request by adding the environment variable `SM_CHANNEL_TRAINING` as the default value for the `--data_dir` argument:

```python
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # reads input channels training and testing from the environment variables
  parser.add_argument('--data_dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
```

Script mode displays the list of available environment variables in the training logs. You can find the [entire list here](https://github.com/aws/sagemaker-containers/blob/master/README.rst#list-of-provided-environment-variables-by-sagemaker-containers).

# Training in SageMaker

After you test the training job locally, upload the dataset to an S3 bucket so SageMaker can access the data during training:

In [7]:
import sagemaker

inputs = sagemaker.Session().upload_data(path='/home/ec2-user/SageMaker/albert-chinese-ner/data', key_prefix='DEMO-tensorflow-albert-chinese-ner')
print(inputs)

s3://sagemaker-cn-northwest-1-343958593302/DEMO-tensorflow-albert-chinese-ner


The returned variable inputs above is a string with a S3 location which SageMaker Tranining has permissions
to read data from.

To train in SageMaker:
- change the estimator argument `train_instance_type` to any SageMaker ml instance available for training.
- set the `training` channel to a S3 location.

In [8]:
estimator = TensorFlow(entry_point='albert_ner.py',
                       source_dir='.',
                       train_instance_type='ml.p3.2xlarge', # Executes training in a ml.p2.xlarge instance
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.15.2',
                       py_version='py3',
                       script_mode=True)

estimator.fit({'training': inputs})

2020-05-28 10:36:56 Starting - Starting the training job...
2020-05-28 10:37:00 Starting - Launching requested ML instances......
2020-05-28 10:37:58 Starting - Preparing the instances for training......
2020-05-28 10:39:18 Downloading - Downloading input data...
2020-05-28 10:39:40 Training - Downloading the training image.....WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2020-05-28 10:40:36,263 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-05-28 10:40:40,262 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
   

## Git Support

In [ ]:
git_config = {'repo': 'https://github.com/whn09/albert-chinese-ner.git', 'branch': 'master'}

# TODO You need to add codes to albert_ner.py to download and extract data

estimator = TensorFlow(entry_point='albert_ner.py',
                       source_dir='.',
                       git_config=git_config,
                       train_instance_type='ml.p2.xlarge', # Executes training in a ml.p2.xlarge instance
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.15.2',
                       py_version='py3',
                       script_mode=True)

estimator.fit({'training': inputs})